In [1]:
%cd assets

/home/mrokita/pobr-ffmpeg/assets


# FFprobe

In [2]:
!ffprobe -hide_banner video2.mp4

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'video2.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.76.100
  Duration: 00:00:07.92, start: 0.000000, bitrate: 2839 kb/s
  Stream #0:0(jpn): Video: h264 (High 10) (avc1 / 0x31637661), yuv420p10le(tv, bt709), 1920x1080 [SAR 1:1 DAR 16:9], 2513 kb/s, 23.98 fps, 23.98 tbr, 24k tbn, 47.95 tbc (default)
    Metadata:
      handler_name    : Video
      vendor_id       : [0][0][0][0]
  Stream #0:1(jpn): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, 5.1, fltp, 342 kb/s (default)
    Metadata:
      handler_name    : Audio
      vendor_id       : [0][0][0][0]


<video controls src="assets/video2.mp4" width=500px />

# FFMpeg - wykrywanie krawędzi

In [4]:
!ffmpeg -hide_banner -i video2.mp4 -vf edgedetect=low=0.1:high=0.5 -y outputs/canny_edge.mp4

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'video2.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.76.100
  Duration: 00:00:07.92, start: 0.000000, bitrate: 2839 kb/s
  Stream #0:0(jpn): Video: h264 (High 10) (avc1 / 0x31637661), yuv420p10le(tv, bt709), 1920x1080 [SAR 1:1 DAR 16:9], 2513 kb/s, 23.98 fps, 23.98 tbr, 24k tbn, 47.95 tbc (default)
    Metadata:
      handler_name    : Video
      vendor_id       : [0][0][0][0]
  Stream #0:1(jpn): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, 5.1, fltp, 342 kb/s (default)
    Metadata:
      handler_name    : Audio
      vendor_id       : [0][0][0][0]
Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> h264 (libx264))
  Stream #0:1 -> #0:1 (aac (native) -> aac (native))
Press [q] to stop, [?] for help
[libx264 @ 0x55a8cedf28c0] using SAR=1/1
[libx264 @ 0x55a8cedf28c0] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 

<video controls src="assets/outputs/canny_edge.mp4" width=500px />

In [5]:
!ffmpeg -hide_banner -i video2.mp4 \
      -filter_complex "[0]format=gbrp,histogram=display_mode=stack[1];\
      [1]scale=iw*2:ih[hist];\
      [0]edgedetect,scale=iw/4:ih/4[edges];\
      [0][hist]overlay[hist_over];\
      [hist_over][edges]overlay=x=1000:y=0" \
      -y outputs/histogram.mp4

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'video2.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.76.100
  Duration: 00:00:07.92, start: 0.000000, bitrate: 2839 kb/s
  Stream #0:0(jpn): Video: h264 (High 10) (avc1 / 0x31637661), yuv420p10le(tv, bt709), 1920x1080 [SAR 1:1 DAR 16:9], 2513 kb/s, 23.98 fps, 23.98 tbr, 24k tbn, 47.95 tbc (default)
    Metadata:
      handler_name    : Video
      vendor_id       : [0][0][0][0]
  Stream #0:1(jpn): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, 5.1, fltp, 342 kb/s (default)
    Metadata:
      handler_name    : Audio
      vendor_id       : [0][0][0][0]
Stream mapping:
  Stream #0:0 (h264) -> format (graph 0)
  Stream #0:0 (h264) -> edgedetect (graph 0)
  Stream #0:0 (h264) -> overlay:main (graph 0)
  overlay (graph 0) -> Stream #0:0 (libx264)
  Stream #0:1 -> #0:1 (aac (native) -> aac (native))
Press [q] to stop, [?] for help
[libx264 @ 0x55699e640cc0] 

<video controls src="assets/outputs/histogram.mp4" width=500px />

## ffmpeg-python

(https://github.com/kkroening/ffmpeg-python)

In [12]:
import ffmpeg
    
input_video = ffmpeg.input("video2.mp4")

hist = input_video.filter('format', 'gbrp').filter('histogram', display_mode='stack').filter('scale', 'iw*2', 'ih')

edges = input_video.filter('edgedetect').filter('scale', 'iw/4', 'ih/4')

hist_overlay = input_video.overlay(hist)


out_stream = hist_overlay.overlay(edges, x=1000, y=0)
print(' '.join(out_stream.output(f'outputs/histogram_python.mp4').compile()))

out_stream.output(f'outputs/histogram_python.mp4').overwrite_output().run()

ffmpeg -i video2.mp4 -filter_complex [0]format=gbrp[s0];[s0]histogram=display_mode=stack[s1];[s1]scale=iw*2:ih[s2];[0][s2]overlay=eof_action=repeat[s3];[0]edgedetect[s4];[s4]scale=iw/4:ih/4[s5];[s3][s5]overlay=eof_action=repeat:x=1000:y=0[s6] -map [s6] outputs/histogram_python.mp4


ffmpeg version 4.4.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-D_FORTIFY_SOURCE=2 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 ' --extra-cflags=' -I/usr/include/rav1e' --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libvo-amrwbenc --enable-version3 --enable-bzlib --enable-chromaprint --disable-cr

(None, None)

# Przycinanie pliku

In [7]:
!ffmpeg -hide_banner -i drive.mp4 -ss 00:00:30 -t 10 outputs/drive_trim.mp4

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'drive.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.76.100
  Duration: 00:02:30.08, start: 0.000000, bitrate: 1961 kb/s
  Stream #0:0(eng): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt709), 1920x800 [SAR 1:1 DAR 12:5], 1830 kb/s, 23.98 fps, 23.98 tbr, 24k tbn, 47.95 tbc (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
  Stream #0:1(eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 125 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]
Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> h264 (libx264))
  Stream #0:1 -> #0:1 (aac (native) -> aac (native))
Press [q] to stop, [?] for help
[libx264 @ 0x55d2c6842a00] using SAR=1/1
[libx264 @ 0x55d2c6842a00] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[li

<video controls src="assets/outputs/drive_trim.mp4" width=500px />

# FFMpeg + OpenCV + Python

In [8]:
import ffmpeg
import numpy as np
import cv2

width = 1920
height = 800

process1 = (
    ffmpeg
    .input("outputs/drive_trim.mp4")
    .output('pipe:', format='rawvideo', pix_fmt='bgr24')
    .run_async(pipe_stdout=True)
)

process2 = (
    ffmpeg
    .input('pipe:', format='rawvideo', pix_fmt='gray8', s='{}x{}'.format(width, height))
    .output("outputs/drive_numpy.mp4", pix_fmt='yuv420p')
    .overwrite_output()
    .run_async(pipe_stdin=True)
)

while in_bytes := process1.stdout.read(width * height * 3):
    in_frame = (
        np
        .frombuffer(in_bytes, np.uint8)
        .reshape([height, width, 3]).astype(np.uint8)
    )
    out_frame = in_frame.copy()
    out_frame = cv2.Canny(out_frame, 50, 150)
    process2.stdin.write(
        out_frame.astype(np.uint8)
        .tobytes()
    )

process2.stdin.close()
process1.wait()
process2.wait()


ffmpeg version 4.4.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-D_FORTIFY_SOURCE=2 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 ' --extra-cflags=' -I/usr/include/rav1e' --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libvo-amrwbenc --enable-version3 --enable-bzlib --enable-chromaprint --disable-cr

frame=  240 fps= 72 q=-0.0 Lsize= 1080000kB time=00:00:10.01 bitrate=883852.1kbits/s speed=2.98x    
video:1080000kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000000%
frame=  240 fps= 66 q=-1.0 Lsize=    9230kB time=00:00:09.48 bitrate=7976.1kbits/s speed= 2.6x    
video:9226kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.040983%
[libx264 @ 0x55bb3f5ee680] frame I:8     Avg QP:21.53  size: 81562
[libx264 @ 0x55bb3f5ee680] frame P:60    Avg QP:27.89  size: 49185
[libx264 @ 0x55bb3f5ee680] frame B:172   Avg QP:31.19  size: 33974
[libx264 @ 0x55bb3f5ee680] consecutive B-frames:  3.8%  0.8%  3.8% 91.7%
[libx264 @ 0x55bb3f5ee680] mb I  I16..4: 21.0% 56.2% 22.8%
[libx264 @ 0x55bb3f5ee680] mb P  I16..4:  1.4%  4.2%  9.5%  P16..4:  4.4%  2.8%  2.5%  0.0%  0.0%    skip:75.2%
[libx264 @ 0x55bb3f5ee680] mb B  I16..4:  0.4%  0.7%  3.4%  B16..8: 10.1%  4.1%  3.3%  direct: 1.1%  skip:77.0%  L0:52.0% L1:41.3% BI: 6.7%
[libx264 @ 0

0

<video controls src="assets/outputs/drive_numpy.mp4" width=500px />

# Śledzenie postępu

In [9]:
import socket
from threading import Thread
from contextlib import contextmanager
import tempfile
from pathlib import Path


@contextmanager
def open_progress_socket():
    with tempfile.TemporaryDirectory() as tempdir:
        sock = socket.socket(socket.AF_UNIX, socket.SOCK_STREAM)
        socket_filename = Path(tempdir) / "progress.sock"
        sock.bind(str(socket_filename))
        sock.settimeout(15)
        try:
            sock.listen(1)
            yield sock, socket_filename
        finally:
            sock.close()
            

def watch_progress(sock: socket.socket):
        conn, _ = sock.accept()
        while rec := conn.recv(4096):
            out_time_us = dict(
                map(
                    lambda x: x.split("="),
                    rec.decode("utf-8").split("\n")[:-1],
                ))["out_time_us"]
            downloaded_seconds = round(int(out_time_us) / 1000000)
            print(f"Przetworzono {downloaded_seconds} sekund")
            
with open_progress_socket() as (sock, socket_filename):
    total_duration = float(ffmpeg.probe("outputs/drive_trim.mp4")["streams"][0]["duration"])
    print(f"Całkowity czas trwania pliku: {total_duration}")
    t = Thread(target=watch_progress, args=(sock,))
    t.start()
    (
        ffmpeg
        .input("outputs/drive_trim.mp4")
        .output("outputs/drive_h265.mp4", vcodec="hevc")
        
        .overwrite_output()
        .global_args(
            "-progress",
            f"unix://{socket_filename}"
        )
    ).run(quiet=True)
    t.join()

Całkowity czas trwania pliku: 10.01
Przetworzono 0 sekund
Przetworzono 2 sekund
Przetworzono 3 sekund
Przetworzono 4 sekund
Przetworzono 5 sekund
Przetworzono 7 sekund
Przetworzono 8 sekund
Przetworzono 9 sekund
Przetworzono 10 sekund
Przetworzono 10 sekund
